## Download Data First

In [ ]:
!curl -O -J -L https://www.bodc.ac.uk/data/open_download/gebco/GEBCO_30SEC/zip/

In [ ]:
!unzip GEBCO_2014.zip

In [ ]:
!curl -O -J -L https://cchdo.ucsd.edu/data/14186/33RO20161119_hy1.csv >> 33RO20161119_hy1.csv

In [ ]:
import scipy.interpolate as scint
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.32.5.238:34809")
client

# Load data

In [ ]:
# topography
ds = xr.open_dataset('GEBCO_2014_2D.nc',decode_times=False)
ds

In [ ]:
# remove comments from observation csv
f = open('33RO20161119_hy1.csv','r')
lines = f.readlines()
f = open('33RO20161119_hy1.csv','w')
f.writelines([line for line in lines if '#' not in line])

In [ ]:
# obsrvations
df = pd.read_csv('33RO20161119_hy1.csv',skiprows=[0,2],na_values=-999)
df.columns

In [ ]:
df.head()

In [ ]:
ovar = 'SALNTY' # observed variable we want to plot
station_header_name = 'STNNBR'
lat_header_name = 'LATITUDE'
lon_header_name = 'LONGITUDE'
depth_header_name = 'CTDPRS'

In [ ]:
df = df.dropna(subset=[depth_header_name, station_header_name])

# Interpolate Observations to Mesh

In [ ]:
# create a mesh
depth = np.linspace(0,6000,100)
stations = df.groupby(station_header_name).mean().index #groupby by station
lat_station = df.groupby(station_header_name).mean()[lat_header_name]
lon_station = df.groupby(station_header_name).mean()[lon_header_name]

stations_grid,depth_grid = np.meshgrid(stations,depth)

In [ ]:
# you should probably check that these variables are not loaded in as strings
stations_obs = df[station_header_name]
lat_obs = df[lat_header_name]
lon_obs = df[lon_header_name]
depth_obs = df[depth_header_name]
ovar_obs = df[ovar] 

# interp obs to mesh for plotting
interpolated_obs = scint.griddata((stations_obs, depth_obs/10),ovar_obs,
                          (stations_grid, depth_grid/10),
                          method='linear')

interpolated_obs = xr.DataArray(interpolated_obs,dims=['depth','station'],
                                coords={'depth':('depth',-depth),
                                        'station':('station',stations),
                                        'lat':('station',lat_station),
                                        'lon':('station',lon_station)})

# Unmasked Section

In [ ]:
interpolated_obs.plot()

# Mask Topography

Use xarray's interpolation function to subsample topography

In [ ]:
lat = xr.DataArray(lat_station, dims='station')
lon = xr.DataArray(lon_station, dims='station')
topo = ds.elevation.interp(lon=lon, lat=lat)

In [ ]:
plt.pcolormesh(interpolated_obs.lat,
               interpolated_obs.depth,
               interpolated_obs.where(interpolated_obs.depth > topo),
               vmin=34,vmax=35.5)

plt.title(ovar)
plt.xlabel('Latitude')
plt.ylim(-5000,0)
plt.colorbar()

# Let's see how well this works

### Scatter plot of raw observations

In [ ]:
plt.figure(dpi=300)

plt.scatter(lat_obs, -depth_obs, s=0.5, c=ovar_obs,
           vmin=34,vmax=35.5,cmap=cmocean.cm.haline)

plt.title(ovar)
plt.xlabel('Latitude')
plt.ylim(-5000,0)
plt.colorbar()

### Gray points indicate where samples were taken. The blank spots are where there isn't enough data to interpolate

In [ ]:
plt.figure(dpi=300)

plt.pcolormesh(interpolated_obs.lat,
               interpolated_obs.depth,
               interpolated_obs.where(interpolated_obs.depth > topo),
               vmin=34,vmax=35.5)

plt.scatter(lat_obs, -depth_obs, s=0.25, c='0.5',
           vmin=34,vmax=35.5)

plt.title(ovar)
plt.xlabel('Latitude')
plt.ylim(-5000,0)
plt.colorbar()

### Here we compare the raw observations (colored points) to the interpolated observations. If you can't see the point, the interpolation is doing a good job

In [ ]:
plt.figure(dpi=300)

plt.pcolormesh(interpolated_obs.lat,
               interpolated_obs.depth,
               interpolated_obs.where(interpolated_obs.depth > topo),
               vmin=34,vmax=35.5)

plt.scatter(lat_obs, -depth_obs, s=0.25, c=ovar_obs,
           vmin=34,vmax=35.5)

plt.title(ovar)
plt.xlabel('Latitude')
plt.ylim(-5000,0)
plt.colorbar()